# Data Collection

In [2]:
!pip install nbimporter wrds

  Using cached nbimporter-0.3.4-py3-none-any.whl (4.9 kB)
  Using cached wrds-3.1.1-py3-none-any.whl (12 kB)
  Using cached psycopg2_binary-2.9.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)


In [3]:
import nbimporter
from credentials import WRDS_USERNAME
import pandas as pd
import wrds

# Stock datatype, sectors is a list of tuples with S&P sector information
from sector import Stock, sectors

In [74]:
# Authenticate and cache credentials
db = wrds.Connection(wrds_username=WRDS_USERNAME)

Enter your WRDS username [rjsmithers3]: 
Enter your password: ···············


WRDS recommends setting up a .pgpass file.


Create .pgpass file now [y/n]?:  n


You can create this file yourself at any time
with the create_pgpass_file() function.
Loading library list...
Done


In [75]:
START_DT = '2015-01-01'

sp5 = Stock('SP500', '', 'S&P 500 Index')
sp5.data = db.raw_sql("SELECT 'SP500' as ticker, 'S&P 500 Index' as descr, caldt as date,spindx as close,sprtrn as retx FROM crsp.dsp500 WHERE caldt > '{}'".format(START_DT))
# Can use the get_table function, however this is limited on our SQL input 
# data = db.get_table(library='crsp', table='dsp500', columns = ['caldt', 'spindx', 'sprtrn'])

stock_data = [sp5]

for desc, ticker, cusip in sectors:
    sector = Stock(ticker, cusip, desc)
    sector.data = db.raw_sql("SELECT '{}' as ticker, '{}' as descr, date, bidlo as low, askhi as high, prc as close, vol, ret, bid, ask, retx FROM crsp.dsf WHERE cusip = '{}' AND date > '{}'".format(ticker, desc, cusip, START_DT))
    stock_data.append(sector)


In [76]:
# db.list_tables(library="crsp")
# db.describe_table('crsp', 'msf')

In [77]:
# Terminate our connection for data gathering
db.close()

In [78]:
# Compute HIGH, LOW from data (needed for technical analysis)

all_data = []
for stock in stock_data:
    all_data.append(stock.data)

joined = pd.concat(all_data)

In [79]:
# Save our data
for stock in stock_data:
    stock.data.to_csv("data/sectors/{}_7yr_daily.csv".format(stock.ticker))

In [80]:
joined

,ticker,descr,date,close,retx,low,high,vol,ret,bid,ask
0,SP500,S&P 500 Index,2015-01-02,2058.200000,-0.000340,NaN,NaN,NaN,NaN,NaN,NaN
1,SP500,S&P 500 Index,2015-01-05,2020.580000,-0.018278,NaN,NaN,NaN,NaN,NaN,NaN
2,SP500,S&P 500 Index,2015-01-06,2002.610000,-0.008893,NaN,NaN,NaN,NaN,NaN,NaN
3,SP500,S&P 500 Index,2015-01-07,2025.900000,0.011630,NaN,NaN,NaN,NaN,NaN,NaN
4,SP500,S&P 500 Index,2015-01-08,2062.140000,0.017888,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1758,XLU,Utilities,2021-12-27,70.059998,0.004877,69.540001,70.059998,7853685.0,0.004877,70.050003,70.059998
1759,XLU,Utilities,2021-12-28,70.690002,0.008992,70.110001,70.709999,8052724.0,0.008992,70.699997,70.709999
1760,XLU,Utilities,2021-12-29,71.070000,0.005376,70.650002,71.180000,6877849.0,0.005376,71.059998,71.070000
1761,XLU,Utilities,2021-12-30,71.330002,0.003658,70.809998,71.400002,9336214.0,0.003658,71.330002,71.339996


In [81]:
joined.to_csv("data/sp5_sectors_7yr_daily.csv")